# Introduction
## Why did we do this?
Many of the IQMs calculated are "no-reference" metrics. "A no-reference IQM is a measurement of some aspect of the actual image which cannot be compared to a reference value for the metric since there is no ground-truth about what this number should be."

Our goal was provide context for the image quality metrics (IQMs) shown in the MRIQC group reports, by showing the distribution of IQMs for your data plotted relative to a larger set of anonymized IQMs pulled from the web API.

## What MRIQCeption does
mriqception takes user IQMs from MRIQC and plots them relative to IQMs pulled from the 200k+ images in MRIQC web API (we're going to call those "normative" IQMs). The user has the option to filter their API query by relevant acquisition parameters, such as TR/TE.

mriqception also features a brief description of the IQM, shown as a tooltip when you mouseover the name of the IQM. We have tried to make these descriptions as user-friendly as possible.

In [2]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
#define widgets
modality_widget=widgets.RadioButtons(
    options=['bold', 'structural'],
    description='Modality:',
    disabled=False
)

TR_min=widgets.FloatSlider(
    min=1.5,
    max=5,
    step=0.1,
    description='TR min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)

In [4]:
TR_max=widgets.FloatSlider(
    min=1.5,
    max=4,
    step=0.1,
    description='TR max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)



TE_min=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


In [5]:
TE_max=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


select_parameters=widgets.SelectMultiple(
    options=['TR_min', 'TR_max', 'TE_min', 'TE_max'],
    description='Parameters',
    disabled=False)

In [6]:
display(modality_widget)

RadioButtons(description='Modality:', options=('bold', 'structural'), value='bold')

In [7]:
modal=modality_widget.value
display(select_parameters)
print("If empty, no parameter restrictions will applied")

SelectMultiple(description='Parameters', options=('TR_min', 'TR_max', 'TE_min', 'TE_max'), value=())

If empty, no parameter restrictions will applied


In [8]:
for i in select_parameters.value:
        display(eval(i))

FloatSlider(value=1.5, continuous_update=False, description='TR max:', max=4.0, min=1.5, readout_format='')

In [9]:
to_filter=select_parameters.value
filter_list=[]
filter_dict={'TR_min': "TR >= {}".format(TR_min.value),
             'TR_max': "TR < {}".format(TR_max.value),
             'TE_min': "TE >= {}".format(TE_min.value),
             'TE_max': "TE < {}".format(TE_max.value)
            }

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)
    
print(filter_list)
#filter_list= ['TR > 2.2', 'TR < 3']

['TR < 2.6']


In [10]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# path to input of local data from MRIQC on your own dataset 
group_file = os.path.join(here,'test_data', 'group_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   Tesla, TE, TR
   NOTE: Only working as *and* right now!
"""


# IQM variables to visualize
#need to add separate IQMs for structural and functional
IQM_to_plot = ['aor','aqi','dummy_trs','dvars_nstd','dvars_std','dvars_vstd',
                    'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
                    'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
                    'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
                    'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
                    'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
                    'summary_fg_p95','summary_fg_stdv','tsnr']

In [11]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

(18, 45)

In [15]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
if not filter_list == []:
    filtered_apidf = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(list(filtered_apidf))
filtered_apidf.head()

(1000, 77)
(834, 77)


,_id,summary_fg_n,fber,dummy_trs,gcor,fwhm_x,gsr_y,summary_fg_mean,_created,aor,...,bids_meta_session_id,bids_meta_ConversionSoftware,bids_meta_EffectiveEchoSpacing,bids_meta_PhaseEncodingDirection,bids_meta_ScanningSequence,bids_meta_CogAtlasID,bids_meta_SliceEncodingDirection,bids_meta_ParallelReductionFactorInPlane,bids_meta_PulseSequenceType,bids_meta_TotalReadoutTime
17,59a4e0c5265d200008a1da3f,25274,1.979839e+07,0,0.042622,2.618985,0.012357,601.891602,"Tue, 29 Aug 2017 03:34:29 GMT",0.003113,...,dbeb1b75b6d3b0a6e4c6b4237ac54b3704547069faf30b...,GE_dcm_to_nii.sh,0.00032,j,NaN,NaN,NaN,NaN,NaN,NaN
18,59a4e11e265d200008a1da42,25347,1.523840e+07,0,0.025999,2.622077,0.013856,689.687378,"Tue, 29 Aug 2017 03:35:58 GMT",0.009450,...,e6f64de1bd538ad0374810eea8993dd1110d1bc552d910...,GE_dcm_to_nii.sh,0.00032,j,NaN,NaN,NaN,NaN,NaN,NaN
19,59a4e152265d2000079e882c,25956,4.088632e+06,0,0.125848,2.550383,0.007899,931.122253,"Tue, 29 Aug 2017 03:36:50 GMT",0.001761,...,dbeb1b75b6d3b0a6e4c6b4237ac54b3704547069faf30b...,GE_dcm_to_nii.sh,0.00032,j,NaN,NaN,NaN,NaN,NaN,NaN
20,59a4f375265d200008a1da4c,25847,1.914398e+03,0,0.024652,2.334496,0.031287,810.450134,"Tue, 29 Aug 2017 04:54:13 GMT",0.003170,...,10ccd191d2d0cf162dd07594ee4e7932034aa84693eed8...,Dimon,0.00056,j,NaN,NaN,NaN,NaN,NaN,NaN
21,59a50df1265d200008a1da60,29851,2.338218e+05,0,0.036474,2.881513,0.019061,1167.772583,"Tue, 29 Aug 2017 06:47:13 GMT",0.015904,...,dbeb1b75b6d3b0a6e4c6b4237ac54b3704547069faf30b...,GE_dcm_to_nii.sh,0.00032,j,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape


(852, 78)

In [14]:
# Visualization # 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"",outliers=True)

widgets.VBox([v[0],v[1]])


Loading in dataframe...
Loading variables: ['aor', 'aqi', 'dummy_trs', 'dvars_nstd', 'dvars_std', 'dvars_vstd', 'efc', 'fber', 'fd_mean', 'fd_num', 'fd_perc', 'fwhm_avg', 'fwhm_x', 'fwhm_y', 'fwhm_z', 'gcor', 'gsr_x', 'gsr_y', 'snr', 'summary_bg_k', 'summary_bg_mad', 'summary_bg_mean', 'summary_bg_median', 'summary_bg_n', 'summary_bg_p05', 'summary_bg_p95', 'summary_bg_stdv', 'summary_fg_k', 'summary_fg_mad', 'summary_fg_mean', 'summary_fg_median', 'summary_fg_n', 'summary_fg_p05', 'summary_fg_p95', 'summary_fg_stdv', 'tsnr']
Loading in data descriptors...
